In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import zipfile

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1DcMia8gTGt7sFcRJl2WCfaoXAdmxaFRP -O /tmp/data.zip

In [0]:
local_zip = "/tmp/data.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
base_dir = "/tmp/data"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

In [0]:
train_green_dir = os.path.join(train_dir, "green")
train_half_dir = os.path.join(train_dir, "half_burnt")
train_burnt_dir = os.path.join(train_dir, "burnt")
validation_green_dir = os.path.join(validation_dir, "green")
validation_half_dir = os.path.join(validation_dir, "half_burnt")
validation_burnt_dir = os.path.join(validation_dir, "burnt")

In [0]:
!ls /tmp/data
num_green_tr = len(os.listdir(train_green_dir))
num_half_tr = len(os.listdir(train_half_dir))
num_burnt_tr = len(os.listdir(train_burnt_dir))

num_green_val = len(os.listdir(validation_green_dir))
num_half_val = len(os.listdir(validation_half_dir))
num_burnt_val = len(os.listdir(validation_burnt_dir))

total_train = num_burnt_tr + num_green_tr + num_half_tr
total_val = num_burnt_val + num_green_val + num_half_val

In [0]:
print("total training green images:", num_green_tr)
print("total training half images:", num_half_tr)
print("total training burnt images:", num_burnt_tr)

print("total validation green images:", num_green_val)
print("total validation half images:", num_half_val)
print("total validation burnt images:", num_burnt_val)

print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [0]:
batch_size = 10
epochs = 90
IMG_SHAPE = 400

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)
custom_image_generator = ImageDataGenerator(rescale=1./255)

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, 
                                                     directory=train_dir, 
                                                     shuffle=True, 
                                                     seed=123,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode="categorical")

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size, 
                                                              directory=validation_dir, 
                                                              target_size=(IMG_SHAPE,IMG_SHAPE),
                                                              class_mode="categorical")

In [0]:
sample_training_images, _ = next(train_data_gen) 

In [0]:
model = Sequential()
model.add(Conv2D(16, 3, padding="same", activation="relu", input_shape=(IMG_SHAPE,IMG_SHAPE, 3,))) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, 3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, 3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, 3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [0]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"]
             )

In [0]:

#model training
#comment out this block for imported model
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(batch_size))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(batch_size)))
)


acc = history.history["acc"]
val_acc = history.history["val_acc"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()


In [0]:
#Uncomment the folowing lines to import the model that I have trained earlier also comment out the model training code block

#!gdown https://drive.google.com/uc?id=1Whs2JT6pbrdOqcpFdAffPtSCcCWR0YSB -O /tmp/model.h5
#model = load_model("/tmp/model.h5")

In [0]:
#comment out this block for imported model

acc = history.history["acc"]
val_acc = history.history["val_acc"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()


In [0]:
test_loss, test_acc = model.evaluate_generator(train_data_gen)
print("Train dataset test accuracy percentage:", test_acc * 100,"%")
print("Train dataset test accuracy decimal:", test_acc)